In [ ]:
import pandas as pd
import numpy as np
from functions import *
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Table of contents**<a id='toc0_'></a>    
- [Combining data sources](#toc1_)    
    - [Add industry data for DK](#toc1_1_1_)    
    - [Add financial data](#toc1_1_2_)    
    - [Add beta](#toc1_1_3_)    
  - [Add risk-free rate](#toc1_2_)    
- [Create variables](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Combining data sources](#toc0_)

In [42]:
trade = pd.read_csv('data/trade.csv', header=[0,1], index_col=0)
stocks = pd.read_csv('data/stocks.csv')
dk_industry = pd.read_csv('data/dk_industry.csv')
financials = pd.read_csv('data/financials.csv')
beta = pd.read_csv('data/beta.csv')
rf_rate = pd.read_csv('data/rf_rate.csv')

In [43]:
# offsets
quarterly_offset = 1 # quarterly data is 2 months behind end of quarter
annual_offset = 6 # annual data is 6 months behind publication

In [44]:
# convert the index to datetime (the index holds the dates)
trade.index = pd.to_datetime(trade.index)
trade = trade.stack(level=0).reset_index()


# rename columns to have a proper
trade.rename(columns={'level_1': 'ticker'}, inplace=True)

# Data Cleaning and Sorting
trade.drop_duplicates(inplace=True)
trade.dropna(inplace=True) # happens if there was one data point the first day of a given ticker but not the rest of the values (e.g. trade values but no ask or bid)
trade.sort_values(['ticker', 'timestamp'], inplace=True)
trade.reset_index(drop=True, inplace=True)


/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_65389/654452550.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  trade = trade.stack(level=0).reset_index()


In [45]:
df = pd.merge(trade, stocks[['ticker','shares','NACE']], how='left', on=['ticker'])
# display(df)


### <a id='toc1_1_1_'></a>[Add industry data for DK](#toc0_)

In [46]:
dk_industry['timestamp'] = pd.to_datetime(dk_industry['timestamp'])
# join stocks and dk_industry on 'NACE industry' 
industry = stocks[['ticker','NACE']].merge(dk_industry, how='left', on='NACE')

# adjust the timestamp to be 2 months behind
industry['timestamp'] = industry['timestamp'] + pd.DateOffset(months=2+quarterly_offset)
industry['timestamp'] = (
    industry['timestamp']
      .dt.to_period('M')
      .dt.to_timestamp('M')
)

industry = industry.drop(columns=['NACE'])

In [47]:
df = pd.merge(df, industry, how='left', on=['timestamp', 'ticker'])

#ffil the industry values
for col in industry.columns[2:]:
    df[col] = df[col].groupby(df['ticker']).ffill()


### <a id='toc1_1_2_'></a>[Add financial data](#toc0_)

In [48]:
financials['timestamp'] = pd.to_datetime(financials['timestamp'])

financials['timestamp'] = financials['timestamp'] + pd.DateOffset(months=annual_offset)
financials['timestamp'] = (
    financials['timestamp']
      .dt.to_period('M')
      .dt.to_timestamp('M')
)

# expand the dataset
financials = (
    financials
        # sort data and find the next timestamp
      .sort_values(['ticker','timestamp'])
      .assign(
        next_fye   = lambda df: df.groupby('ticker')['timestamp'].shift(-1),
        plus_12m   = lambda df: df['timestamp'] + pd.DateOffset(months=12),
        period_end = lambda df: pd.to_datetime(np.where(
                          (df.next_fye - df.timestamp).abs()
                            < 
                          (df.plus_12m   - df.timestamp).abs(),
                          df.next_fye,
                          df.plus_12m
                        ))
      )
      # expand the data
      .assign(timestamp = lambda df: df.apply(expand_monthly, axis=1))
      .explode('timestamp')
      .drop(columns=['next_fye','plus_12m','period_end'])
      .reset_index(drop=True)
)

In [49]:
# join df and financials
df = pd.merge(df, financials, how='left', on=['timestamp', 'ticker'])

# display(df)

### <a id='toc1_1_3_'></a>[Add beta](#toc0_)

In [50]:
beta['timestamp'] = pd.to_datetime(beta['timestamp'])

# adjust to be last trading day of the month only and set to end of month
monthly_beta = (
    beta
    .groupby(
        ['ticker', pd.Grouper(key='timestamp', freq='ME')],
         as_index=False
    )['beta']
    .last()
)

In [51]:
# join df and beta
df = pd.merge(df, monthly_beta, how='left', on=['timestamp', 'ticker'])

## <a id='toc1_2_'></a>[Add risk-free rate](#toc0_)

In [52]:
rf_rate['timestamp'] = pd.to_datetime(rf_rate['timestamp'])
# set date as index and resample by month, taking the mean of diskonto & folio
monthly_avg = (
    rf_rate
    .set_index('timestamp')
    .resample('ME')[['diskonto', 'folio']]
    .mean()
    .rename_axis('timestamp')
    .reset_index()
)
# convert to monthly rates
monthly_avg['diskonto'] = monthly_avg['diskonto'].apply(lambda x: (1 + x) ** (1/12) - 1)
monthly_avg['folio'] = monthly_avg['folio'].apply(lambda x: (1 + x) ** (1/12) - 1)
monthly_avg
# drop diskonto and rename folio to risk-free
monthly_avg.drop(columns=['diskonto'], inplace=True)
monthly_avg.rename(columns={'folio': 'risk_free'}, inplace=True)

In [53]:
# join df and beta
df = pd.merge(df, monthly_avg, how='left', on=['timestamp'])

# display(df)

# <a id='toc2_'></a>[Create variables](#toc0_)

In [ ]:
df['target'] = df.groupby('ticker')['adjclose'].transform(lambda x: x.pct_change(periods=1))

# subtract the risk-free rate from the target
df['target'] = df['target'] - df['risk_free']
df['target'] = df['target'].shift(-1) # shift the target by 1 month

# momentum
momentum_periods = {'mom1m': 1, 'mom3m': 3, 'mom6m': 6, 'mom12m': 12}

# compute percentage change over each period for each ticker separately.
for feature_name, period in momentum_periods.items():
    df[feature_name] = df.groupby('ticker')['adjclose'].transform(lambda x: x.pct_change(periods=period))

# bid-ask spread in percentage
df['baspread'] = (df['ask']-df['bid'])/df['ask']

# drop if mom12m or target is nan (first 12 months and last month for each ticker)
df.dropna(subset=['mom12m','target'], inplace=True)

[2061]
[256]


In [15]:
# drop if any nan
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# save the data
df.to_csv('data/data.csv', index=False)